In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np 
from config import params
from functions_preprocessing import *
from matplotlib import pyplot as plt
import os
from datetime import datetime

#### Import the school data as a Dataframe

In [ ]:
raw_school_df = pd.read_csv(params["school_csv_path"])
raw_school = dict()
for key in params["school_data_dictionary"].keys():
    raw_school[key] = raw_school_df[params["school_data_dictionary"][key]]

school_df = pd.DataFrame(data = raw_school)
preprocess_school_list(school_df)

#### Preprocess the Responses and Add the Hospital-Derived data
This is the preprocessing script for the responses. It will extract the features and remove duplicates. The hospital data will also be preprocessed here. The school data is not yet incorporated and dropped.

In [ ]:
# use the config file to convert the survey questions to shorter names for data processing downstream
response_df = pd.read_csv(params["raw_data_file_path"])
response_df = extract_features(response_df, params)

#remove empty responses and duplicates
response_df =  response_df[response_df[["Email"]].notna().any(axis=1)] #remove responses w/o an email
check_for_duplicates(response_df, params)

In [ ]:
#Extract the hospital data and then drop the hospital feature
derive_hospital_zipcode_county(response_df)
derive_patient_ses(response_df, params)
drop_hospital_name_county_zipcode(response_df)

In [ ]:
#add school data and then drop emails
add_school_data_to_responses(response_df, school_df)
delete_inappropriate_schools(response_df)
one_hot_encode_comms_update_composition(response_df)
drop_incomplete(response_df)

In [ ]:
drop_school_email_data(response_df)
fr_free_df = make_fr_free_version(response_df)

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d%m%Y_%H%M%S")

response_df.reset_index(drop = True, inplace = True)
fr_free_df.reset_index(drop = True, inplace = True)

response_df.to_csv(os.path.join(params["output_base_directory"], "preprocessed_responses-" + dt_string  + ".csv"))
fr_free_df.to_csv(os.path.join(params["output_base_directory"], "preprocessed_responses_no_FR-" + dt_string + ".csv"))

In [ ]:
os.path.join(params["output_base_directory"], "preprocessed_responses-" + dt_string  + ".csv")